# Predicting the Sale Price of Bulldozers using Machine Learning

In this notebook, we're going to go through an example machine learning project with the goal of predicting the sale price of bulldozers 

## 1. Problem Definition 

> How well can we predict the future sale price of a bulldozer, give it's characteristics and previous examples of how much similar bulldozers have been sold for?

## 2. Data
The data is downloaded from the Kaggle Bluebook for bulldozers competition: https://www.kaggle.com/competitions/bluebook-for-bulldozers/data

The data for this competition is split into three parts:

* Train.csv is the training set, which contains data through the end of 2011.
* Valid.csv is the validation set, which contains data from January 1, 2012 - April 30, 2012 You make predictions on this set throughout the majority of the competition. Your score on this set is used to create the public leaderboard.
* Test.csv is the test set, which won't be released until the last week of the competition. It contains data from May 1, 2012 - November 2012. Your score on the test set determines your final rank for the competition.


## 3. Evaluation 

The evaluation metric for this competition is the RMSLE (root mean squared log error) between the actual and predicted auction prices.

For more on the evaluation of this project check: https://www.kaggle.com/competitions/bluebook-for-bulldozers/overview

**Note**: The goal for most regression evaluation metrics is to minimize the error. For example, our goal for this project will be to build a machine learning model which minimises RSMLE.


## 4.Features

Kaggle provides a data dictionary: https://www.kaggle.com/competitions/bluebook-for-bulldozers/data

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import sklearn

In [ ]:
# Import the data training & validation

df = pd.read_csv("./data/TrainAndValid.csv", low_memory=False)

In [ ]:
df.info()

In [ ]:
df.isna().sum()

In [ ]:
fig, ax = plt.subplots()

ax.scatter(df["saledate"][:1000], df["SalePrice"][:1000])


In [ ]:
df.saledate[:1000]

In [ ]:
#df["SalePrice"].hist()  # taking too long

## Parsing Dates 

When we work with time series data, we want to enrich the time & date component as much as possible. 

We can do that by telling pandas which of our columns has dates in it using the `parse_dates` parameter

In [ ]:
# Import data again but this time parse dates 
df = pd.read_csv("./data/TrainAndValid.csv", low_memory=False, parse_dates =["saledate"])

In [ ]:
df.saledate.dtype

In [ ]:
df.saledate[:1000]

In [ ]:
fig, ax = plt.subplots()

ax.scatter(df["saledate"][:1000], df["SalePrice"][:1000])

In [ ]:
df.head()

In [ ]:
df.head().T

In [ ]:
df.saledate.head(20)

### Sort DataFrame by saledate 

When working with Time Series Data it is a good idea to sort it by date.

In [ ]:
df.sort_values(by=["saledate"], inplace = True, ascending=True)

In [ ]:
df.saledate.head(20)

In [ ]:
df.head()

### Make Copy of the original DataFrame 

We make a copy of the original dataframe so when we manipulate the copy, we have still got our original data 

In [ ]:
# Make a copy 
df_temp = df.copy()

In [ ]:
df_temp.saledate.head(20)

### Add datetime parameters for `saledate` column 

In [ ]:
df_temp["saleYear"] = df_temp.saledate.dt.year
df_temp["saleMonth"] = df_temp.saledate.dt.month
df_temp["saleDay"] = df_temp.saledate.dt.day
df_temp["saleDayOfWeek"] = df_temp.saledate.dt.dayofweek
df_temp["saleDayOfYear"] = df_temp.saledate.dt.dayofyear

In [ ]:
df_temp.head().T

In [ ]:
# Now we have enriched our DataFrame with date time features we can remove saledate 
df_temp.drop("saledate",axis=1,inplace=True)

In [ ]:
# Check the values of diffrent columns 
df_temp.state.value_counts()

## 5. Modelling 

We've done enough EDA (we could always do more)  but let's start to do some model driven EDA. 

In [ ]:
# Let's build a machine learning model
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_jobs=-1,
                             random_state = 42 #same like random_seed
                             )

#model.fit(df_temp.drop("SalePrice",axis = 1), df_temp.SalePrice) # is not working because we have some features which datatypes are object 

### Convert string to categories 

One way we can turn all of our data into numbers is by converting them into panda categories 

We can check the opportunities here: https://pandas.pydata.org/docs/reference/api/pandas.Categorical.dtype.html

In [ ]:
pd.api.types.is_string_dtype(df_temp["UsageBand"])

In [ ]:
# Find the columns which contain strings
for label, content in df_temp.items():
    if pd.api.types.is_string_dtype(content):
        print(label)

In [ ]:
# if you are wondering what df.items does here is a example 

random_dict = {"key1":"Hello",
               "key2":"World", }



In [ ]:
for label, content in random_dict.items():
        print(label)

In [ ]:
# This will turn all of the string values into category values 

for label, content in df_temp.items():
    if pd.api.types.is_string_dtype(content):
        df_temp[label] = content.astype("category").cat.as_ordered()

In [ ]:
# Wir wandeln diese Spalte(n) in den pandas-Datentyp "category" um.
# Dabei speichert pandas nicht jeden String-Wert pro Zeile, sondern:
#   1) eine feste Liste aller möglichen Kategorien (Labels) und
#   2) pro Zeile nur einen integer Code, der auf die Kategorie zeigt.
# Vorteil: weniger Speicherverbrauch und oft schnellere Operationen bei wenigen, häufig wiederholten Werten.
# Optional (falls gesetzt): Mit ordered=True bekommt die Kategorie eine feste Reihenfolge, die Sortierung/Vergleiche beeinflusst.

In [ ]:
df_temp.info()

In [ ]:
df_temp.state.cat.categories

In [ ]:
df_temp.state.value_counts()

In [ ]:
# This will turn all of the object values into category values 

for label, content in df_temp.items():
    if pd.api.types.is_object_dtype(content):
        df_temp[label] = content.astype("category").cat.as_ordered()

In [ ]:
# Wir wandeln diese Spalte(n) in den pandas-Datentyp "category" um.
# Dabei speichert pandas nicht jeden String-Wert pro Zeile, sondern:
#   1) eine feste Liste aller möglichen Kategorien (Labels) und
#   2) pro Zeile nur einen integer Code, der auf die Kategorie zeigt.
# Vorteil: weniger Speicherverbrauch und oft schnellere Operationen bei wenigen, häufig wiederholten Werten.
# Optional (falls gesetzt): Mit ordered=True bekommt die Kategorie eine feste Reihenfolge, die Sortierung/Vergleiche beeinflusst.

In [ ]:
df_temp.info()

In [ ]:
df_temp.state.cat.codes

Thanks to pandas Categories we now have a way to access all of our data in the form of numbers.

But we still have a bunch of missing data 

In [ ]:
df_temp.isnull().sum()/len(df_temp)

### Save preprocessed data

In [ ]:
#Export current temp df 

df_temp.to_csv("data/train_temp.csv", index = False)

In [ ]:
# Import preprocessed data 

df_temp=pd.read_csv("data/train_temp.csv", low_memory= False)

df_temp.head().T

In [ ]:
df_temp.isnull().sum()/len(df_temp) 

In [ ]:
df_temp.isna().sum()

## Fill missing values

### Fill numerical missing values first

In [ ]:
df_temp.info()

In [ ]:
for label,content in df_temp.items(): 
    if pd.api.types.is_object_dtype(content): # if read_csv we need to change the dtype again to categorical
        df_temp[label] = content.astype("category").cat.as_ordered()
    if pd.api.types.is_string_dtype(content):
        df_temp[label] = content.astype("category").cat.as_ordered()
    if pd.api.types.is_numeric_dtype(content):
        print(label)

# Hinweis: CSV-Dateien speichern keine Datentyp-Informationen (nur Werte als Text).
# Beim pd.read_csv() werden die Spaltentypen deshalb von pandas neu "erraten" (Type Inference),
# wodurch z.B. category, datetime oder Strings mit führenden Nullen als andere dtypes eingelesen werden können.
# Lösung: dtypes/parse_dates beim Import explizit setzen oder ein Format wie Parquet/Feather nutzen, das dtypes mit speichert.

In [ ]:
df_temp.ModelID

In [ ]:
# Check for which numeric columns have null values 

for label,content in df_temp.items():
    if pd.api.types.is_numeric_dtype(content):
        if pd.isnull(content).sum():
            print(label)
        

In [ ]:
# Fill numeric rows with the median 

for label,content in df_temp.items():
    if pd.api.types.is_numeric_dtype(content):
        if pd.isnull(content).sum():
            #Add a binary column which tells us if the data was missing 
            df_temp[label+"_is_missing"] = pd.isnull(content) 
            #Fill missing numeric values with median 
            df_temp[label]= content.fillna(content.median())

# Mean (arithm. Mittelwert) = Summe aller Werte / Anzahl: nutzt jede Beobachtung direkt und wird durch Ausreißer stark beeinflusst.
# Median = der mittlere Wert der sortierten Daten (bei gerader Anzahl: Mittel der zwei mittleren): robust gegenüber Ausreißern
# und oft besser für schiefe Verteilungen (z.B. Einkommen, Preise, Wartezeiten).

In [ ]:
# Check for which numeric columns have null values 

for label,content in df_temp.items():
    if pd.api.types.is_numeric_dtype(content):
        if pd.isnull(content).sum():
            print(label)

In [ ]:
df_temp.auctioneerID_is_missing.value_counts()

In [ ]:
df_temp.isna().sum()

In [ ]:
### Filling and turning categorical variables into numbers 


#Check for columns which arent't numeric 

for label,content in df_temp.items():
    if not pd.api.types.is_numeric_dtype(content): 
        print(label) 

In [ ]:
pd.Categorical(df_temp["state"]).dtype

In [ ]:
pd.Categorical(df_temp["state"]).codes # if missing value then -1

In [ ]:
# Turn categorical variables into number and fill missing 

for label,content in df_temp.items():
    if not pd.api.types.is_numeric_dtype(content): 
        # add binary columns to indicate whether sample had missing value 
        df_temp[label+"_is_missing"] = pd.isnull(content)
        #Turn categories into numbers and add +1 
        df_temp[label] = pd.Categorical(content).codes+1

In [ ]:
df_temp.head()

In [ ]:
df_temp.info()

In [ ]:
df_temp.head().T

In [ ]:
df_temp.isna().sum()[:70]

 Now that all of our data is numeric as well as our df has no missing values, we should be able to build a machine learning model

In [ ]:
df_temp.head()

In [ ]:
%%time 

#JN Function to calculate how much time the cell take
#Instantiate model
model = RandomForestRegressor(n_jobs = -1)

# Fit the model

model.fit(df_temp.drop("SalePrice", axis = 1), df_temp["SalePrice"])


In [ ]:
# Score the model with train data wrong
model.score(df_temp.drop("SalePrice", axis = 1), df_temp["SalePrice"])

**Question:**
Why doesn't the above metric reliable? 

### Splitting data into train/validation sets

In [ ]:
df_temp.saleYear

In [ ]:
df_temp.saleYear.value_counts()

In [ ]:
# 1) Split We have data leakage because we calculate the median with past and future datasets
df_val   = df_temp[df_temp.saleYear == 2012].copy()
df_train = df_temp[df_temp.saleYear != 2012].copy()

# 2) Imputation-Statistiken nur aus Training
num_cols = df_train.select_dtypes(include="number").columns
medians = df_train[num_cols].median()

df_train[num_cols] = df_train[num_cols].fillna(medians)

# 2) Imputation-Statistiken nur aus Validation
num_cols = df_val.select_dtypes(include="number").columns
medians = df_val[num_cols].median()

df_val[num_cols] = df_val[num_cols].fillna(medians)

In [ ]:
# Split data into X and y 

X_train,y_train = df_train.drop("SalePrice",axis = 1), df_train.SalePrice

# Split data into X and y 

X_valid,y_valid = df_val.drop("SalePrice",axis = 1), df_val.SalePrice

### Build an evaluation fuction 


<img src="./images/metric.png"/>

In [ ]:
# Create evaluation function (the competition uses RMSLE) 
from sklearn.metrics import mean_squared_log_error, mean_absolute_error,r2_score

def rmsle (y_test,y_preds):
    """
    Calculates root mean squared log error between predictions and true labels. 
    """
    return np.sqrt(mean_squared_log_error(y_test,y_preds))

# Create a function to evaluate model on a few diffrent levels 
def show_scores(model):
    train_preds = model.predict(X_train)
    val_preds = model.predict(X_valid)
    scores = {"Training MAE":mean_absolute_error(y_train,train_preds),
    "Valid MAE": mean_absolute_error(y_valid,val_preds),
    "Training RMSLE":rmsle(y_train,train_preds),
    "Valid RMSLE":rmsle(y_valid,val_preds),
    "Training r^2":r2_score(y_train,train_preds),
    "Valid r^2":r2_score(y_valid,val_preds)}
    return scores

## Testing our model on a subset (to tune the hyperparameters)

In [ ]:
model = RandomForestRegressor(n_jobs = -1,
                              random_state = 42) 

#This takes far too long .... for experimenting
#model.fit(X_train,y_train)

In [ ]:
len(X_train)

In [ ]:
#Change max_samples value 

model = RandomForestRegressor(n_jobs = -1, random_state = 42,max_samples=10000)

In [ ]:
%%time
##cutting down on the max number of samples each estimators can see improves training time
model.fit(X_train,y_train)

In [ ]:
show_scores(model)

### Hyperparameter tuning with RandomizedSearchCV

In [ ]:
%%time
from sklearn.model_selection import RandomizedSearchCV

#Diffrent RandomForestRegressor HP 
rf_grid ={
    "n_estimators":np.arange(10,100,10),
    "max_depth":[None,3,5,10],
    "min_samples_split": np.arange(2,20,2),
    "min_samples_leaf":np.arange(1,20,2),
    "max_samples":[10000],
    "max_features":[0.5,1,"sqrt","auto"]
}

# Instantiate RandomizesSearchCV 
rs_model = RandomizedSearchCV(RandomForestRegressor(n_jobs = -1, random_state = 42), param_distributions=rf_grid,
                              n_iter= 5, # very low because it take too long! 
                              cv=5, 
                              verbose = True)

#Fit the RSCV model
rs_model.fit(X_train,y_train)

In [ ]:
#find the best HP for the model 

rs_model.best_params_

In [ ]:
# Evaluate the randomized Search Model 
show_scores(rs_model)

### Train the model with the best hyperparameters 

**Note** These were found after 100 iterations of RandomizedSearchCV.

In [ ]:
%%time 

# Most ideal Hyperparameters 

ideal_model = RandomForestRegressor(n_estimators = 40,
                                    min_samples_leaf=1,
                                    min_samples_split=14,
                                    max_features=0.5,
                                    n_jobs = -1,
                                    max_samples = None,
                                   random_state = 42 # so our results are reproducable
)

#Fit the ideal model 
ideal_model.fit(X_train,y_train) 

In [ ]:
show_scores(ideal_model)

## Make Predictions on test data 

In [ ]:
#Import the test data 

df_test = pd.read_csv("data/Test.csv", low_memory = False, parse_dates=["saledate"]) 

df_test.head()


In [ ]:
df_test.isna().sum()

In [ ]:
df_test.dtypes

In [ ]:
df_test.columns

## Preprocessing the data to the same format like X_Train 

In [ ]:
def preprocessing_data(df):
    """
    Performs transformation on df and returns transformed df. 
    """
    df["saleYear"] = df.saledate.dt.year
    df["saleMonth"] = df.saledate.dt.month
    df["saleDay"] = df.saledate.dt.day
    df["saleDayOfWeek"] = df.saledate.dt.dayofweek
    df["saleDayOfYear"] = df.saledate.dt.dayofyear

    df.drop("saledate", axis = 1, inplace = True)
    
    # Fill the numeric rows with median 
    for label,content in df.items():
        if pd.api.types.is_numeric_dtype(content):
            if pd.isnull(content).sum():
            #Add a binary column which tells us if the data was missing 
                df[label+"_is_missing"] = pd.isnull(content) 
            #Fill missing numeric values with median 
                df[label]= content.fillna(content.median())

    #Filled categorical missing datat and turned categories into numbers 

        if not pd.api.types.is_numeric_dtype(content):
            df[label+"_is_missing"] = pd.isnull(content) 
        #We add +1 to the category code because pandas encodes missing categories with -1 
            df[label]=pd.Categorical(content).codes+1
    
    return df 

In [ ]:
df_test = preprocessing_data(df_test)

df_test.columns, df_test.info()

In [ ]:
# Make predictions on updated Test data dont work because a feature is missing
#test_preds = ideal_model.predict(df_test)

In [ ]:
# We can find how the columns differ using sets
set(X_train.columns)-set(df_test.columns)

In [ ]:
# Manually fit the column 
df_test["auctioneerID_is_missing"] = False

df_test.head()

In [ ]:
# nach model.fit(X_train, y_train)
fit_cols = ideal_model.feature_names_in_

# X_test / X_new exakt wie beim Fit ausrichten
X_new_aligned = df_test.reindex(columns=fit_cols, fill_value=0)



test_preds = ideal_model.predict(X_new_aligned)

In [ ]:
# We can find how the columns differ using sets
set(X_train.columns)-set(df_test.columns)

In [ ]:
len(test_preds)

In [ ]:
test_preds

In [ ]:
# Format  predictions into the same format Kaggle is after 

df_preds = pd.DataFrame()
df_preds ["SalesID"] = df_test["SalesID"]
df_preds ["SalesPrice"] = test_preds

df_preds


In [ ]:
df_preds.to_csv("data/test_predictions.csv", index = False) 

### Feature Importance

seeks to figure out which diffrent attributes of the data were most importance when it comes to predicting the target variable

In [ ]:
# FInd feature importance of our best models 

len(ideal_model.feature_importances_)

In [ ]:
# Helper function for plotting feature importance
def plot_features(columns, importanceFactor, top_n=20):
    columns = np.array(columns)
    importanceFactor = np.array(importanceFactor)

    # sortiere nach Wichtigkeit (absteigend) und nimm Top-N
    idx = np.argsort(importanceFactor)[::-1][:top_n]
    cols = columns[idx]
    imps = importanceFactor[idx]

    fig, ax = plt.subplots(figsize=(10, 6))
    ax.barh(cols[::-1], imps[::-1])  # barh + umdrehen => wichtigste oben
    ax.set_xlabel("Feature importance")
    ax.set_title(f"Top {top_n} Feature Importances")
    plt.tight_layout()
    plt.show()

plot_features(X_new_aligned.columns, ideal_model.feature_importances_, top_n=20)
    


In [ ]:
plot_features(X_new_aligned.columns,ideal_model.feature_importances_)

In [ ]:
def plot_feature_other_way (columns, importances, n = 20):
    df = (pd.DataFrame({"features":columns, 
                        "features_importances": importances})
          .sort_values("features_importances",ascending = False) 
          .reset_index(drop=True)) 

    # PLot the dataframe 
    fig, ax = plt.subplots() 
    #ax.barh(np.flip(np.array(df["features"][:n:])), np.flip(np.array(df["features_importances"][:n:])))
    ax.barh(df["features"][:n:][::-1], df["features_importances"][:n:][::-1])
    ax.set_xlabel("Feature importance")
    ax.set_title(f"Top {n} Feature Importances")
    plt.tight_layout()
    plt.show()

In [ ]:
plot_feature_other_way(X_new_aligned.columns,ideal_model.feature_importances_)

**Question to finish:**  Why might knowing the feature importances of a trained machine learning model be helpful?

**Final challenge:** What other machine learning models could you try on our dataset? Hint checkout the regression section of scikit learn map or try to look at CatBoost.ai or XGBoost.ai